<a href="https://colab.research.google.com/github/BenxiaHu/DeepSilencer/blob/master/CNN_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# Define a toy dataset (replace with your own dataset)
# For this example, we'll use random sequences and labels
num_sequences = 2000
sequence_length = 200
num_classes = 2

sequences = np.random.choice(['A', 'C', 'G', 'T'], size=(num_sequences, sequence_length))
labels = np.random.randint(0, num_classes, size=num_sequences)

# Convert DNA sequences to one-hot encoding
def one_hot_encoding(sequences):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot_sequences = np.zeros((len(sequences), 4, len(sequences[0])))
    for i, seq in enumerate(sequences):
        for j, base in enumerate(seq):
            one_hot_sequences[i, mapping[base], j] = 1
    return one_hot_sequences

one_hot_sequences = one_hot_encoding(sequences)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(one_hot_sequences, labels, test_size=0.2, random_state=42)

# Define a CNN-Transformer model
class CNNTransformer(nn.Module):
    def __init__(self, input_size, d_model, num_encoder_layers, num_classes, nhead=8):
        super(CNNTransformer, self).__init()
        self.cnn_block = CNNBlock(input_size)

        # TransformerEncoderLayer and nn.Embedding
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.embedding = nn.Embedding(4, d_model)  # 4 represents A, C, G, T
        self.fc = nn.Sequential(
            nn.Linear(d_model, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        cnn_output = self.cnn_block(x)

        # Apply nn.Embedding
        x = self.embedding(cnn_output.permute(2, 0, 1).long())

        # Transformer encoding
        transformer_output = self.transformer_encoder(x)
        avg_pooled = torch.mean(transformer_output, dim=0)
        output = self.fc(avg_pooled)
        return output

# CNN Block with Residual Connection
class CNNBlock(nn.Module):
    def __init__(self, input_size):
        super(CNNBlock, self).__init__()
        self.conv1 = nn.Conv1d(input_size, 64, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(2)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        # Implement residue connection
        out = out + self.conv2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        return out

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for training and testing
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNTransformer(input_size=4, d_model=128, num_encoder_layers=6, num_classes=num_classes, nhead=8).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

num_epochs = 10

model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')
